In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import pymongo
import time
from tqdm import tqdm
import schedule 

In [ ]:
def extract_tweets():
    hashtags_data = pd.read_excel(r'Hashtags_csv.xlsx')
    hashtags_df = pd.DataFrame(hashtags_data)
    tweets = []
    for h in tqdm(hashtags_df['Mots clés']):
        scraper = sntwitter.TwitterSearchScraper(h).get_items()
        m = 100
        for i, tweet in enumerate(scraper):
            if tweet.lang == 'fr':
                data = [
                    tweet.id,
                    tweet.date,
                    tweet.rawContent,
                    tweet.user.username,
                    tweet.likeCount,
                    tweet.retweetCount,
                    h
                ]
                tweets.append(data)
                if i >= m:
                    break
    tweets_df = pd.DataFrame(tweets, columns=['id', 'date', 'contenu', 'utilisateur', 'nbre_like', 'nbre_retweet', 'mots_cles'])
    tweets_db = tweets_df.to_dict(orient='records')
    tweets_df.to_json('tweets.json', orient='records')

    client = pymongo.MongoClient("mongodb://localhost:27017")
    db = client["TweetsScraping"]
    db.Tweets.insert_many(tweets_db)

In [ ]:
# Planification de l'exécution de la fonction toutes les 8 heures
schedule.every(8).hours.do(extract_tweets)

while True:
    schedule.run_pending()
    time.sleep(1)

In [ ]:
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client["TweetsScraping"]
db.Tweets.insert_many(tweets_db)